## Requests

In [1]:
import requests, json, csv, time, urllib3
from urllib.parse import urlencode, quote_plus, unquote
from pymongo import MongoClient
import os
import pandas as pd
import pickle
import json
import sqlite3
from random import *
from datetime import datetime
import surprise
from surprise.model_selection import KFold
from surprise import Reader, Dataset
import numpy as np
from surprise import SVD, evaluate
from surprise import NMF, KNNBasic



"""
Date : 2018-08-22

Author : Hwang Taelim
Content :
- Class for data connect with GET request // DataConnect

"""

class DataConnect():
    def __init__(self, url, headers, params=None):
        self.url = url
        self.params = params
        self.headers = headers
        self.data = ""

    def getData(self, num_retries=5):
        with requests.Session() as s:
            s.keep_alive = False
            try:
                res = s.get(url=self.url, params = self.params, headers=self.headers, timeout=10)
    #         res = requests.get(url=self.url, params = self.params, headers=self.headers, timeout=1)
                if 500 <= res.status_code<600:
                    print(html.status_code, res, reason)
                    raise ConnectionError

            except:
                s.close()
                print("--TRY CONNECTION {0}...--".format(num_retries))
                if num_retries > 0:
#                     time.sleep(60)
                    return self.getData(num_retries = (num_retries-1))
                else:
                    s.close()
                    print("--MAX RETRY REACHED!--")
                    return None
            else:
                resStr = res.content
                resStr = resStr.decode('utf-8')
                resObj = json.loads(resStr)
        #         print(json.dumps(resObj, indent="  ",  ensure_ascii=False))
                self.data = resObj
                s.close()
                return resObj


    def setUrl(self, url):
        self.url = url

    def setParams(self, params):
        self.params = params

    def setHeaders(self, headers):
        self.headers = headers

class APIConnect():
    def __init__(self, api_list):
        self.api_list = api_list
        self.reconnect_url = []

    def seoul_api(self, db_conn):

        for api in self.api_list:
            if "_w_" in api:
                db_conn.set_db("OpenAPIWithPos")
            else:
                db_conn.set_db("OpenAPIWithoutPos")
            db_conn.set_item("seoul")
            with open(api, 'r', newline='') as f:
                f_csv = csv.reader(f, delimiter='\t')
                url_list = []

                for row in (row for row in f_csv if row[0] == 'seoul'):
                    url_list.append(row)

                for url_data in url_list:
                    if 'http' in url_data[3]:
                        # Initial data getter
                        url_origin = url_data[3]+"/"+url_data[5]+"/json/"+url_data[4]
                        url_init = url_origin+"/1/1"
#                         print(url_init)
                        headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
                                "Connection": "close",
                                }
                        data_connect = DataConnect(url=url_init, headers=headers)
                        res_data = data_connect.getData()


                        try:
                            res_data[url_data[4]]
                        except Exception as e:
                            print(e)
                            print(res_data["RESULT"]["CODE"], ":", res_data["RESULT"]["MESSAGE"])
                        else:
                            # Seoul API의 category data를 입력한다
                            db_conn.update_category(res_data, "seoul")
                            divisor = 250
                            data_count = res_data[url_data[4]]["list_total_count"]
                            loop_count = data_count // divisor
                            loop_remain = data_count % divisor
                            print("--LOAD {0} ITEMS, SEPERATED BY 1000 ({1}, {2})--".format(data_count, loop_count, loop_remain))
                            for i in range(loop_count):
                                url_req = url_origin+"/"+str(i*divisor+1)+"/"+str((i+1)*divisor)
                                print(url_req)
                                data_connect.setUrl(url_req)
                                res_data = data_connect.getData()
                                # 이후 DB에 넣음
                                db_conn.update_seoul(res_data, i*divisor)
                            if loop_remain > 0:
                                url_req = url_origin+"/"+str(loop_count*divisor+1)+"/"+str((loop_count*divisor)+loop_remain)
                                print(url_req)
                                data_connect.setUrl(url_req)
                                res_data = data_connect.getData()
                                # 이후 DB에 넣음
                                db_conn.update_seoul(res_data, loop_count*divisor)
                            # print(data_count)


    def data_go_api(self, db_conn, api_name, params):

        for api in self.api_list:
            if "_w_" in api:
                db_conn.set_db("OpenAPIWithPos")
            else:
                db_conn.set_db("OpenAPIWithoutPos")
            db_conn.set_item("data_go")
            with open(api, 'r', newline='') as f:
                f_csv = csv.reader(f, delimiter='\t')
                url_list = []

                for row in (row for row in f_csv if row[0] == 'data_go'):
                    url_list.append(row)

                for url_data in url_list:
                    if 'http' in url_data[3] and api_name == url_data[4]:
                        # Initial data getter
                        url_origin = url_data[3]+"/"+api_name
                        url_init = url_origin
                        print(url_init)
#                         print(url_init)
                        headers={
                            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
                            'Connection':'close',
                                }

                        try:
                            data_connect = DataConnect(url=url_init, params = params, headers=headers)
                            res_data = data_connect.getData()
                            res_data["body"]["items"]
                        except:
                            print(res_data["header"]["resultCode"], ":", res_data["header"]["resultMsg"])
                        else:
                            # 이후 DB에 넣음
                            db_conn.update_data_go(res_data, api_name)
                            # print(data_count)


class DBConnect():
    def __init__(self, host="127.0.0.1", port=27017, db="admin"):
        print("--INIT MONGO DB CONNECTION--")
        self.client = MongoClient(host = host, port = port)
        self.db = self.client.get_database(db)
        self.item = self.db['item']
        print("--CLIENT : {0}--".format(self.client))
        print("--DB : {0}--".format(self.db))
        print("--ITEM : {0}--".format(self.item))


    def set_item(self, item="item"):
        self.item = self.db[item]

    def set_db(self, db):
        self.db = self.client.get_database(db)

    def get_item_by_cat(self, search_key):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.find({
            'cat_name': search_key
        },
            {
                '_id': False
            })
        return list(result)

    def get_item_by_title(self, search_key):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.find({
            'items.title': search_key
        },
            {
                '_id': False
            })
        return list(result)
    
    def get_item_category(self):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.find({
        },
            {
#                 'cat_id':True,
                'items':False,
                'pop_tags':False,
                '_id': False
            })
        return list(result)
    
    def get_item_all(self):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.find({
        },
            {
#                 'cat_id':True,
                '_id': False
            })
        return list(result)

    def get_item_by_latlon(self, search_key):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.find({
            "$or": [{'items.lat': search_key[0]}, {'items.lon': search_key[1]}]

        },
            {
                '_id': False
            })
        return list(result)
    
    def set_item_score(self, input_key):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.update_one({
            "cat_id":input_key},
            {'$inc': {'items.star_cnt': 1}
        })

    def set_item_review(self, input_key, input_data):
        self.set_db("map_db")
        self.set_item("place")
        result = self.item.update_one({
            "cat_id":input_key},
            {'$inc': {'items.rev_cnt': input_data}
        })
        
        
    def update_seoul(self, items, index):
        print("--UPDATE START--")
#         print(self.item)
#         print(self.db)
#         print(self.client)
#         print(items[list(items.keys())[0]]["row"])
        if items == None:
            print("--NO ITEMS!--")
            return
        for row in items[list(items.keys())[0]]["row"]:
            try:
#                 print(index, row)
                result = self.item.update_one(
                    {
                        "api_id": list(items.keys())[0]+"_"+str(index)
                    },
                    {
                        '$set': row
                    },
                    upsert=True,
                )
#                 print(result.raw_result)

                index += 1
            except Exception as e:
                print(e)

        print("--DATA UPDATE IS COMPLETED--")

    def update_data_go(self, items, name):
        print("--UPDATE START--")
        index = 0
        items = items["body"]
        if len(items.keys()) == 0:
            print("--NO ITEMS!--")
            return
        for row in items["items"]:
            try:
#                 print(index, row)
                result = self.item.update_one(
                    {
                        "api_id": list(name+"_"+str(index))
                    },
                    {
                        '$set': row
                    },
                    upsert=True,
                )
#                 print(result.raw_result)

                index += 1
            except Exception as e:
                print(e)

        print("--DATA UPDATE IS COMPLETED--")

    def update_csv(self, name, csv_id = ""):
        self.set_db("OpenAPIWithPos")
        self.set_item("csv_"+name)
        print("--UPDATE START--")
        url = "csv/"+name
        file_list = []
        for(dirpath, dirnames, filenames) in os.walk(url):
            file_list.extend(filenames)
            break
        print(file_list)
        if len(file_list) == 0:
            print("--NO ITEMS!--")
            return

        # 일단 다 지우기...
        result = self.item.delete_many({})
        for file in file_list:
            df = pd.read_csv(url+"/"+file, engine='python')
            data = df[df["시도명"] == "서울특별시"]
            data_json = json.loads(data.to_json(orient='records'))
#             print(data_json)
            try:
#                 print(index, row)
                result = self.item.insert_many(
                    data_json,
                )
#                 print(result.raw_result)

            except Exception as e:
                print(e)

        print("--DATA UPDATE IS COMPLETED--")

    def update_crawl(self, name = "/", crawl_id = ""):
        self.set_db("OpenAPIWithPos")

        print("--UPDATE START--")
        url = "crawl"+name
        file_list = []
        for(dirpath, dirnames, filenames) in os.walk(url):
            file_list.extend(filenames)
            break

        print(file_list)
        if len(file_list) == 0:
            print("--NO ITEMS!--")
            return

        # 일단 다 지우기...

        for file in file_list:
            self.set_item("crawl_"+file.split(".")[0])
            result = self.item.delete_many({})
            with open(url+"/"+file, 'rb') as f:
                data = pickle.load(f)
            for i in range(len(data)):
                data[i]["api_id"] = "diningCode_"+str(i)

#                 print(data[i])
            data_json = data

#             print(data_json)
            try:
#                 print(index, row)
                result = self.item.insert_many(

                    data_json,
                )
#                 print(result.raw_result)

            except Exception as e:
                print(e)

        print("--DATA UPDATE IS COMPLETED--")


    def update_map_db(self):
        print("--SET DB POINTER TO OPENAPIWITHPOS--")
#         self.db = self.client.get_database("OpenAPIWithPos")
        self.set_db("OpenAPIWithPos")
        print("--SELECT ITEMS--")

        items = []
        # seoul data
        self.set_item("seoul")
        items.extend(list(self.item.find({})))

        # csv data(우선은 직접 명시)
        self.set_item("csv_small_shop")
        items.extend(list(self.item.find({})))

        # crawl data
        self.set_item("crawl_dining_code")
        items.extend(list(self.item.find({})))



        print("--SET DISCRIMINATOR--")
        disc_api_id = ['api_id', '상가업소번호']
        disc_title = ['title', 'TITLE', 'GIGU', 'GOSU_CD','COURSE_NAME', 'NM', 'LBRRY_NAME', 'NAME_KOR', 'COT_CONTS_NAME', '상호명']
        disc_x = ['lat','X', 'XCODE', 'XCNTS', 'WGS84_Y', 'COT_COORD_Y', '위도']
        disc_y = ['lon','Y', 'YCODE', 'YDNTS', 'WGS84_X', 'COT_COORD_X', '경도']
        disc_text = ['text','CONTENT','REMARK', 'CULTURE_BUSINESS_TYPE', 'CODENAME', '표준산업분류명']
        disc_tag = []
        disc_cat = ['상권업종중분류명']
        disc_addr = ['addr','ADDR', 'ADDR_OLD', 'ADRES', 'ADD_KOR', 'COT_ADDR_FULL_NEW', 'COT_ADDR_FULL_OLD', '도로명주소']
        disc_call = ['call','TEL_NO', 'TEL']
        disc_image = ['img']
        print("--SET DB POINTER TO MAP_DB--")
        self.set_db("map_db")

        print("--SELECT ITEMS--")
        self.set_item("place")

        print("--UPDATE START--")

        for row in items:
            set_api_id = ""
            set_title = ""
            set_text=""
            set_tag=""
            set_cat=""
            set_addr=""
            set_call=""
            set_x = ""
            set_y = ""
            set_image = ""


            for api_id in disc_api_id:
                if api_id in row.keys():
                    set_api_id = row[api_id]
            for title in disc_title:
                if title in row.keys():
                    set_title = row[title]
            for x in disc_x:
                if x in row.keys():
                    set_x = row[x]
            for y in disc_y:
                if y in row.keys():
                    set_y = row[y]

            for text in disc_text:
                if text in row.keys():
                    set_text = row[text]
            for tag in disc_tag:
                if tag in row.keys():
                    set_tag = row[tag]
            for cat in disc_cat:
                if cat in row.keys():
                    set_cat = row[cat]
                if "diningCode" in set_api_id:
                    set_cat = "테이크아웃"
                elif "MgisTraShop" in set_api_id:
                    set_cat = "시장"
                elif "SeoulLibraryTime" in set_api_id:
                    set_cat = "도서관"
                elif "mgis" in set_api_id.lower() or "SebcTourStreetKor" in set_api_id:
                    set_cat = "공원 및 산책길"
            for addr in disc_addr:
                if addr in row.keys():
                    set_addr = row[addr]
            for call in disc_call:
                if call in row.keys():
                    set_call = row[call]
            for image in disc_image:
                if image in row.keys():
                    set_image = row[image]
            insert_data = {
                            'cat_name':set_cat,
                            'pop_tags':set_tag,
                            'items':{
                                'title': set_title,
                                'text':set_text,
                                'star_cnt':0,
                                'rev_cnt':0,
                                'addr':set_addr,
                                'call':set_call,
                                'lon': set_x,
                                'lat': set_y,
                                'image' : set_image
                            }
                }
#             print(insert_data)
            try:
#                 print(index, row)


                result = self.item.update_one(
                    {
                        "cat_id": set_api_id
                    },
                    {
                        '$set': insert_data

                    },
                    upsert=True,
                )
#                 print(result.raw_result)

            except Exception as e:
#                 print(e)
                print("--ERROR!--")
            else:
                pass
#                 print("--ONE ITEM UPDATED--")

        print("--DATA UPDATE IS COMPLETED--")

    def update_category(self, items, name):
        print("--UPDATE START--")
        db = self.client.get_database("OpenAPICategory")
        db_item = db[name]

#         items = items["body"]
#         if len(items.keys()) == 0:
#             print("--NO ITEMS!--")
#             return

        if items == None:
            print("--NO ITEMS!--")
            return
        for row in items[list(items.keys())[0]]["row"]:
            try:
#                 print(index, row)
                result = db_item.update_one(
                    {
                        "api_id": list(items.keys())[0]
                    },
                    {
                        '$set': row

                    },
                    upsert=True,
                )
#                 print(result.raw_result)

            except Exception as e:
                print(e)

            
class RDBConnect():
    def drop_table(self, table):

        # user table
        self.conn.execute("""
            drop table {0}
        """.format(table))
        
    def make_user_table(self):
        
        
        ran_age_list = np.random.choice([10, 20, 30, 40, 50, 60, 70], size=1000, p=[.0262, .4832, .2612, .148, .0584, .0155, .0075])
        ran_sex_list = np.random.choice(['남성', '여성'], size=1000, p=[.62, .38])
        # print(ran_age_list)
        for i in range(1000):
            user_id = i

            ran_age = randint(ran_age_list[i], ran_age_list[i]+9)
            # 연령대는?
            #
            ran_email = "user_"+str(i)+"""@gmail.com"""
            pwd = "1234"

            ran_sex = ran_sex_list[i]
            ran_phone = "010-"+str(randint(1000, 9999))+'-'+str(randint(1000, 9999))
            query = """insert into "user" (user_id, email, password, age, sex, phone_num) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
            .format(user_id, ran_email, pwd, ran_age, ran_sex, ran_phone)
        #     print(query)
            self.cur.execute(
                query
            )

    def make_rate_table(self):

        # key list
        key_list = self.db_conn.get_item_category()
        concat_key_list = []
        for item in key_list:
            concat_key_list.append(item)
            
        cat_list = ['PC/오락/당구/볼링등', '공원 및 산책길', '놀이/여가/취미', '도서관', 
                    '무도/유흥/가무', '스포츠/운동', '시장', '실내운동시설', '연극/영화/극장', 
                    '요가/단전/마사지', '제과제빵떡케익', '커피점/카페', '테이크아웃']
        
        
        

        rate_id = 0
        ran_score_list = np.random.choice([1, 2, 3, 4, 5], size=1000, p=[.05, .10, .7, .10, .05])
        
        
        # print(ran_score_list)
        for i in range(1000):
            ran_range = randint(10, 1000)
            ran_cat = sample(cat_list, 3)
            for j in range(ran_range):
#                 ran_score = ran_score_list[randint(0, 999)]
                place_sample = sample(concat_key_list, 1)[0]
                place_id = place_sample["cat_id"]
                
                if place_sample["cat_name"] in ran_cat:
                    ran_score = 5
                else:
                    ran_score = ran_score_list[randint(0, 999)]
                    
                if ran_score >= 3:
                    ran_review = 1
                else:
                    ran_review = -1
                query = """insert into "rate" (rate_id, user_id, place_id, date, score, review) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
                .format("rate_"+str(rate_id), "user_"+str(i), place_id, datetime.now(), ran_score, ran_review)

                self.cur.execute(
                    query
                )
                # 평가 개수 증가
        #         db_conn.set_item_score(place_id)

                # 리뷰 점수 증가
        #         db_conn.set_item_review(place_id, ran_review)
                rate_id += 1
            
    def close(self):
        self.conn.close()
        
    def __init__(self):

        # mongoDB 연결
        self.db_conn = DBConnect()

        # sqlite 연결
        self.conn = sqlite3.connect("testDB.db")
        self.cur = self.conn.cursor()

        self.cur.execute(
        """
         create table if not exists "user" (
            "user_id" int primary key not null ,
            "email" text not null,
            "password" text not null,
            "age" int,
            "sex" text not null,
            "phone_num" text
            )
        """
        )

        self.cur.execute(
        """
         create table if not exists "rate" (
            "rate_id" int primary key not null ,
            "user_id" int references user(user_id),
            "place_id" text not null,
            "date" text not null,
            "score" int not null ,
            "review" int not null
            )
        """
        )
        self.make_user_table()
        self.make_rate_table()

# user table to pandas

class Recommender(RDBConnect):
    def __init__(self):
        super(Recommender, self).__init__()

        self.cur.execute("""select * from user""")
        result = self.cur.fetchall()
        self.data_user = pd.DataFrame(result)
        self.data_user.columns = ['user_id', 'email', 'password', 'age', 'sex', 'phone_num']

        # rate table to pandas
        self.cur.execute("""select * from rate""")
        result = self.cur.fetchall()
        self.data_rate = pd.DataFrame(result)
        # score는 최종 여정에 대한 평가고...
        # 리뷰 스코어랑 별점 스코어는 장소에 대한 스코어
        self.data_rate.columns = ['rate_id', 'user_id', 'place_id', 'date', 'score', 'review']

        # item table to pandas
        result = self.db_conn.get_item_all()
        self.data_item = pd.DataFrame(result)
        
        # close sqlite3
        self.close()

    def get_data(self, input_key):
        if input_key == "data_user":
            return self.data_user
        elif input_key == "data_rate":
            return self.data_rate
        elif input_key == "data_item":
            return self.data_item

    def make_df(self):

        data = self.data_rate.copy()
        del data["date"]

        data = pd.merge(data, self.data_item, left_on = 'place_id', right_on = 'cat_id')
        del data["cat_id"]
        del data["cat_name"]
        del data["pop_tags"]

        data_list = []
        for row in data["items"]:
            data_list.append(row)

        data_item = pd.DataFrame(data_list)

        new_data = pd.merge(data, data_item, left_on = data.index, right_on = data_item.index)

        del new_data["items"]
        del new_data["key_0"]
        del new_data["addr"]
        del new_data["call"]
        del new_data["image"]
        del new_data["title"]
        del new_data["text"]

        return new_data

    def model(self, alg_key):

        reader = Reader(rating_scale = (1, 5))
        # review	lat	lon	rev_cnt	star_cnt
        data_result = Dataset.load_from_df(self.make_df()[['user_id', 'place_id', 'score']], reader)

        # split data into 5 folds

        data_result.split(n_folds=10)

        # evaluation

        if alg_key.lower() == "svd":
            alg = SVD()
        elif alg_key.lower() == "knn":
            alg = KNNBasic()
        elif alg_key.lower() == "nmf":
            alg = NMF()

        evaluate(alg, data_result, measures=['RMSE', 'MAE'])

        # prediction
        # user_0	smallShop_5645	2
        test_user = 'user_1'
        test_id = 'smallShop_7089'
        real_score = 4

        trainset = data_result.build_full_trainset()

        from collections import defaultdict

        top_n = defaultdict(list)
        testset = trainset.build_anti_testset()
        predictions = alg.test(testset)
        
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))

        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:10]

        print(top_n)
        
        alg.train(trainset)
        
        return alg
    




In [ ]:
if __name__=="__main__":
    api_connect = APIConnect(["url_list_w_lat.txt", "url_list_wo_lat.txt"])
    db_conn = DBConnect("150.95.204.252")
    api_connect.seoul_api(db_conn)
    
#     params={
#         'radius':5000,
#         'cx':127.004528,
#         'cy':37.567538,
#         'type':'json',
#         'ServiceKey':unquote('9yufdGwfG5nTrm48106s%2B%2FQK%2Bz6byu8kQyqGYX7ywOTcSZz5hKnJG6OSAFPymm3Ei6TrKcsL3Osas1zm4v6HmA%3D%3D'),
#     }
#     api_connect.data_go_api(db_conn,"storeZoneInRadius",params)

    db_conn.update_csv("small_shop")
    db_conn.update_crawl()
    db_conn.update_map_db()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['150.95.204.252:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['150.95.204.252:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['150.95.204.252:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--
--UPDATE START--
--LOAD 1291 ITEMS, SEPERATED BY 1000 (5, 41)--
http://openapi.seoul.go.kr:8088/424367587667686b38305544767173/json/SeoulLibraryTime/1/250
--UPDATE START--
--DATA UPDATE IS COMPLETED--
http://openapi.seoul.go.kr:8088/424367587667686b38305544767173/json/SeoulLibraryTime/251/500
--UPDATE START--
--DATA UPDATE IS COMPLETED--
http://openapi.seoul.go.kr:8088/424367587667686b38305544767173/json/SeoulLibraryTime/501/750
--UPDATE START--
--DATA UPDATE IS COMPLETED--
http://openapi.seoul.go.kr:8088/424367587667686b38305544767173/json/SeoulLibraryTime/751/1000
--UPDATE START--


In [2]:
test = Recommender()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [9]:
test.data_rate

,rate_id,user_id,place_id,date,score,review
0,rate_0,user_0,smallShop_13247,2018-08-28 07:26:08.932242,5,1
1,rate_1,user_0,smallShop_20238,2018-08-28 07:26:08.933243,4,1
2,rate_2,user_0,smallShop_14095,2018-08-28 07:26:08.933243,3,1
3,rate_3,user_0,smallShop_24041,2018-08-28 07:26:08.933243,3,1
4,rate_4,user_0,smallShop_26560,2018-08-28 07:26:08.933243,2,-1
5,rate_5,user_0,diningCode_3195,2018-08-28 07:26:08.933243,3,1
6,rate_6,user_0,smallShop_1445,2018-08-28 07:26:08.933243,3,1
7,rate_7,user_0,smallShop_20120,2018-08-28 07:26:08.933243,3,1
8,rate_8,user_0,smallShop_6774,2018-08-28 07:26:08.933243,5,1
9,rate_9,user_0,smallShop_8557,2018-08-28 07:26:08.933243,3,1


In [3]:
result = test.model("knn")

C:\Development Tools\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Development Tools\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0611
MAE:  0.8640
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0586
MAE:  0.8622
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0540
MAE:  0.8568
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0627
MAE:  0.8636
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0608
MAE:  0.8630
------------
Fold 6
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0606
MAE:  0.8615
------------
Fold 7
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0580
MAE:  0.8607
------------
Fold 8
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0582
MAE:  0.8603
---

C:\Development Tools\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Done computing similarity matrix.


In [14]:
result.predict('user_823', 'smallShop_19140', 4)


Prediction(uid='user_823', iid='smallShop_19140', r_ui=4, est=3.2588981337620613, details={'actual_k': 13, 'was_impossible': False})

In [ ]:
result.test()

In [32]:
from collections import defaultdict

top_n = defaultdict(list)
for uid, iid, true_r, est, _ in result:
    top_n[uid].append((iid, est))
    
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:1000]
    
top_n

TypeError: 'KNNBasic' object is not iterable

In [ ]:
db_conn.update_map_db()

--SET DB POINTER TO OPENAPIWITHPOS--
--SELECT ITEMS--
--SET DISCRIMINATOR--
--SET DB POINTER TO MAP_DB--
--SELECT ITEMS--
--UPDATE START--


In [3]:
db_conn = DBConnect()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [ ]:
items = db_conn.update_map_db()

In [ ]:
api_connect = APIConnect(["url_list_w_lat.txt", "url_list_wo_lat.txt"])

In [4]:
db_conn = DBConnect()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [ ]:
api_connect.seoul_api(db_conn)

In [ ]:
params={
    'radius':5000,
    'cx':127.004528,
    'cy':37.567538,
    'type':'json',
    'ServiceKey':unquote('9yufdGwfG5nTrm48106s%2B%2FQK%2Bz6byu8kQyqGYX7ywOTcSZz5hKnJG6OSAFPymm3Ei6TrKcsL3Osas1zm4v6HmA%3D%3D'),
}
api_connect.data_go_api(db_conn, params)

In [ ]:
dataConnect = DataConnect(url="https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do",
                      params={'searchValue' : "0"},
                       headers={
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36"}
                      )

## Mongo DB

In [ ]:
from pymongo import MongoClient

class DBConnect():
    def __init__(self, host="127.0.0.1", port="27017", db="admin"):
        self.client = MongoClient(host = host, port = port)
        self.db = self.client.get_database(db)
        self.item = self.db['item']
    
    def set_db(self, db):
        self.db = self.client.get_database(db)

    def set_item(self, item):
        self.item = self.db[item]
        
    def update_items(self, items):
        try:
            result = self.item.update_one(
                {"title": info_list[1]},
                {
                    '$set':{'img_url':img_url,'img_base64':img_base64}
                },
                upsert=True,
            )
            print(result.raw_result)
        except Exception as e:
            print(e)
            break

In [5]:
result = db_conn.get_item_by_title("고덕산 산책길")

In [7]:
print(list(result))

[{'_id': ObjectId('5b7c9e6958366c5eb551ee6f'), 'cat_id': 'SeoulGilWalkCourse_0', 'cat_name': '', 'items': {'title': '고덕산 산책길', 'text': ' 해발 100m 미만의 낮고 순탄한 숲길이지만 계절 따라 순환하는 자연의 섭리를 그대로 느낄 수 있다. ', 'star_cnt': '', 'rev_cnt': '', 'addr': '', 'call': '', 'lon': '212796.5744', 'lat': '451823.5078', 'image': ''}, 'pop_tags': ''}, {'_id': ObjectId('5b7c9e6958366c5eb551ee71'), 'cat_id': 'SeoulGilWalkCourse_1', 'cat_name': '', 'items': {'title': '고덕산 산책길', 'text': ' 해발 100m 미만의 낮고 순탄한 숲길이지만 계절 따라 순환하는 자연의 섭리를 그대로 느낄 수 있다. ', 'star_cnt': '', 'rev_cnt': '', 'addr': '', 'call': '', 'lon': '212740.8039', 'lat': '450271.8163', 'image': ''}, 'pop_tags': ''}, {'_id': ObjectId('5b7c9e6958366c5eb551ee73'), 'cat_id': 'SeoulGilWalkCourse_2', 'cat_name': '', 'items': {'title': '고덕산 산책길', 'text': ' 해발 100m 미만의 낮고 순탄한 숲길이지만 계절 따라 순환하는 자연의 섭리를 그대로 느낄 수 있다. ', 'star_cnt': '', 'rev_cnt': '', 'addr': '', 'call': '', 'lon': '213032.5589', 'lat': '451967.7418', 'image': ''}, 'pop_tags': ''}, {'_id': ObjectId('5b7

In [1]:
result = db_conn.get_item_by_cat("한식")

NameError: name 'db_conn' is not defined

In [5]:
result.count()

4

In [19]:
result = db_conn.item.find({
    'cat_id':'GolfF
})

In [20]:
result.count()

1

## sqlite

In [4]:
import sqlite3
from random import *
from datetime import datetime
import numpy as np



db_conn = DBConnect()

conn = sqlite3.connect("testDB.db")
cur = conn.cursor()

# user table
conn.execute("""
drop table user
""")

cur.execute(
"""
 create table if not exists "user" (
    "user_id" int primary key not null ,
    "email" text not null,
    "password" text not null,
    "age" int,
    "sex" text not null,
    "phone_num" text 
    )
"""
)

# rate table
conn.execute("""
drop table rate
""")

cur.execute(
"""
 create table if not exists "rate" (
    "rate_id" int primary key not null ,
    "user_id" int references user(user_id),
    "place_id" text not null,
    "date" text not null,
    "score" int not null ,
    "review" int not null
    )
"""
)

ran_age_list = np.random.choice([10, 20, 30, 40, 50, 60, 70], size=1000, p=[.0262, .4832, .2612, .148, .0584, .0155, .0075])
ran_sex_list = np.random.choice(['남성', '여성'], size=1000, p=[.62, .38])
# print(ran_age_list)
for i in range(1000):
    user_id = i
    
    ran_age = randint(ran_age_list[i], ran_age_list[i]+9)
    # 연령대는?
    # 
    ran_email = "user_"+str(i)+"""@gmail.com"""
    pwd = "1234"
    
    ran_sex = ran_sex_list[i]
    ran_phone = "010-"+str(randint(1000, 9999))+'-'+str(randint(1000, 9999))
    query = """insert into "user" (user_id, email, password, age, sex, phone_num) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
    .format(user_id, ran_email, pwd, ran_age, ran_sex, ran_phone)
#     print(query)
    cur.execute(
        query
    )

# key list
key_list = db_conn.get_item_key()
concat_key_list = []
for item in key_list:
    concat_key_list.append(item['cat_id'])    
    
rate_id = 0
ran_score_list = np.random.choice([1, 2, 3, 4, 5], size=1000, p=[.05, .10, .7, .10, .05])
# print(ran_score_list)
for i in range(1000):
    ran_range = randint(1, 100)
    for j in range(ran_range):
        ran_score = ran_score_list[randint(0, 999)]
        place_id = sample(concat_key_list, 1)[0]
        if ran_score >= 3:
            ran_review = 1
        else:
            ran_review = -1
        query = """insert into "rate" (rate_id, user_id, place_id, date, score, review) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
        .format("rate_"+str(rate_id), "user_"+str(i), place_id, datetime.now(), ran_score, ran_review)

        cur.execute(
            query
        )
        # 평가 개수 증가
#         db_conn.set_item_score(place_id)
        
        # 리뷰 점수 증가
#         db_conn.set_item_review(place_id, ran_review)
        rate_id += 1
    


--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [ ]:
db_conn = DBConnect()

In [30]:
result = db_conn.get_item_all()

In [21]:
len(result)

34539

In [22]:
import pickle

with open('result.plk', 'wb') as fp:
    pickle.dump(result, fp)

In [5]:
import surprise
import pandas as pd

# user table to pandas
cur.execute("""select * from user""")
result = cur.fetchall()
data_user = pd.DataFrame(result)
data_user.columns = ['user_id', 'email', 'password', 'age', 'sex', 'phone_num']

# rate table to pandas
cur.execute("""select * from rate""")
result = cur.fetchall()
data_rate = pd.DataFrame(result)
# score는 최종 여정에 대한 평가고...
# 리뷰 스코어랑 별점 스코어는 장소에 대한 스코어
data_rate.columns = ['rate_id', 'user_id', 'place_id', 'date', 'score', 'review']

# item table to pandas
result = db_conn.get_item_all()
data_item = pd.DataFrame(result)

In [6]:
data_item

,cat_id,cat_name,items,pop_tags
0,SeoulLibraryTime_0,도서관,"{'title': 'LH 강남3단지 작은도서관', 'text': '', 'star_...",
1,SeoulLibraryTime_1,도서관,"{'title': '강남구립못골도서관', 'text': '', 'star_cnt':...",
2,SeoulLibraryTime_2,도서관,"{'title': '강남역삼푸른솔도서관', 'text': '', 'star_cnt'...",
3,SeoulLibraryTime_3,도서관,"{'title': '강남한신휴플러스8단지 작은도서관', 'text': '', 'st...",
4,SeoulLibraryTime_4,도서관,"{'title': '강남한양수자인작은씨앗도서관', 'text': '', 'star_...",
5,SeoulLibraryTime_5,도서관,"{'title': '개포1동작은도서관', 'text': '', 'star_cnt':...",
6,SeoulLibraryTime_6,도서관,"{'title': '개포4동작은도서관', 'text': '', 'star_cnt':...",
7,SeoulLibraryTime_7,도서관,"{'title': '국립어린이청소년도서관', 'text': '', 'star_cnt...",
8,SeoulLibraryTime_8,도서관,"{'title': '꿈꾸는작은도서관', 'text': '', 'star_cnt': ...",
9,SeoulLibraryTime_9,도서관,"{'title': '꿈밭작은도서관', 'text': '', 'star_cnt': 4...",


In [72]:
data_item.to_excel('item_db.xlsx', sheet_name='sheet1')

PermissionError: [Errno 13] Permission denied: 'item_db.xlsx'

In [7]:
data_user

,user_id,email,password,age,sex,phone_num
0,0,user_0@gmail.com,1234,49,여성,010-5277-2352
1,1,user_1@gmail.com,1234,22,남성,010-3351-2595
2,2,user_2@gmail.com,1234,23,여성,010-3901-6737
3,3,user_3@gmail.com,1234,44,남성,010-1878-9859
4,4,user_4@gmail.com,1234,55,남성,010-2475-7481
5,5,user_5@gmail.com,1234,42,남성,010-9684-5304
6,6,user_6@gmail.com,1234,21,남성,010-9802-7585
7,7,user_7@gmail.com,1234,22,여성,010-7019-4222
8,8,user_8@gmail.com,1234,20,남성,010-9959-3026
9,9,user_9@gmail.com,1234,30,여성,010-3233-1365


In [17]:
data_rate

,rate_id,user_id,place_id,score,review
0,rate_0,user_0,smallShop_23880,3,1
1,rate_1,user_0,smallShop_16834,4,1
2,rate_2,user_0,smallShop_26243,3,1
3,rate_3,user_0,smallShop_8564,2,-1
4,rate_4,user_0,smallShop_23412,3,1
5,rate_5,user_0,smallShop_27305,3,1
6,rate_6,user_0,smallShop_13647,3,1
7,rate_7,user_0,smallShop_7052,3,1
8,rate_8,user_0,smallShop_11926,3,1
9,rate_9,user_0,smallShop_1812,3,1


In [18]:
import surprise

data = data_rate
# del data["date"]
# 여기는 코드를 좀 수정하기



In [19]:
data = pd.merge(data, data_item, left_on = 'place_id', right_on = 'cat_id')
del data["cat_id"]
del data["cat_name"]
del data["pop_tags"]


In [20]:
data

,rate_id,user_id,place_id,score,review,items
0,rate_0,user_0,smallShop_23880,3,1,"{'title': '더스테이크카페', 'text': '비알콜 음료점업', 'star..."
1,rate_45630,user_922,smallShop_23880,2,-1,"{'title': '더스테이크카페', 'text': '비알콜 음료점업', 'star..."
2,rate_1,user_0,smallShop_16834,4,1,"{'title': '밀알베이커리', 'text': '제과점업', 'star_cnt'..."
3,rate_423,user_7,smallShop_16834,3,1,"{'title': '밀알베이커리', 'text': '제과점업', 'star_cnt'..."
4,rate_29362,user_586,smallShop_16834,3,1,"{'title': '밀알베이커리', 'text': '제과점업', 'star_cnt'..."
5,rate_31990,user_646,smallShop_16834,3,1,"{'title': '밀알베이커리', 'text': '제과점업', 'star_cnt'..."
6,rate_2,user_0,smallShop_26243,3,1,"{'title': '송사부수제쌀고로케', 'text': '제과점업', 'star_c..."
7,rate_3,user_0,smallShop_8564,2,-1,"{'title': '대한노래방업협회중앙회', 'text': '노래연습장 운영업', ..."
8,rate_29019,user_577,smallShop_8564,3,1,"{'title': '대한노래방업협회중앙회', 'text': '노래연습장 운영업', ..."
9,rate_4,user_0,smallShop_23412,3,1,"{'title': '빽다방', 'text': '비알콜 음료점업', 'star_cnt..."


In [24]:
data_list = []
for row in data["items"]:
    data_list.append(row)
    
data_item = pd.DataFrame(data_list)

In [25]:
data_item

,addr,call,image,lat,lon,rev_cnt,star_cnt,text,title
0,서울특별시 송파구 새말로11길 6,,,127.127,37.4836,0,0,비알콜 음료점업,더스테이크카페
1,서울특별시 송파구 새말로11길 6,,,127.127,37.4836,0,0,비알콜 음료점업,더스테이크카페
2,서울특별시 강남구 광평로60길 22,,,127.107,37.487,2,2,제과점업,밀알베이커리
3,서울특별시 강남구 광평로60길 22,,,127.107,37.487,2,2,제과점업,밀알베이커리
4,서울특별시 강남구 광평로60길 22,,,127.107,37.487,2,2,제과점업,밀알베이커리
5,서울특별시 강남구 광평로60길 22,,,127.107,37.487,2,2,제과점업,밀알베이커리
6,서울특별시 양천구 목동중앙북로 38,,,126.867,37.5479,0,0,제과점업,송사부수제쌀고로케
7,서울특별시 관악구 남부순환로191길 12-1,,,126.934,37.486,0,0,노래연습장 운영업,대한노래방업협회중앙회
8,서울특별시 관악구 남부순환로191길 12-1,,,126.934,37.486,0,0,노래연습장 운영업,대한노래방업협회중앙회
9,서울특별시 중구 다산로 131,,,127.011,37.5557,1,1,비알콜 음료점업,빽다방


In [31]:
new_data = pd.merge(data, data_item, left_on = data.index, right_on = data_item.index)

In [36]:
new_data
del new_data["items"]
del new_data["key_0"]
del new_data["addr"]
del new_data["call"]
del new_data["image"]
del new_data["title"]
del new_data["text"]

In [37]:
new_data

,rate_id,user_id,place_id,score,review,lat,lon,rev_cnt,star_cnt
0,rate_0,user_0,smallShop_23880,3,1,127.127,37.4836,0,0
1,rate_45630,user_922,smallShop_23880,2,-1,127.127,37.4836,0,0
2,rate_1,user_0,smallShop_16834,4,1,127.107,37.487,2,2
3,rate_423,user_7,smallShop_16834,3,1,127.107,37.487,2,2
4,rate_29362,user_586,smallShop_16834,3,1,127.107,37.487,2,2
5,rate_31990,user_646,smallShop_16834,3,1,127.107,37.487,2,2
6,rate_2,user_0,smallShop_26243,3,1,126.867,37.5479,0,0
7,rate_3,user_0,smallShop_8564,2,-1,126.934,37.486,0,0
8,rate_29019,user_577,smallShop_8564,3,1,126.934,37.486,0,0
9,rate_4,user_0,smallShop_23412,3,1,127.011,37.5557,1,1


In [92]:
data

,rate_id,user_id,place_id,score,review
0,rate_0,user_0,smallShop_22483,3,1
1,rate_1,user_0,smallShop_1783,2,-1
2,rate_2,user_0,smallShop_16215,3,1
3,rate_3,user_0,smallShop_4389,3,1
4,rate_4,user_0,smallShop_1450,3,1
5,rate_5,user_0,smallShop_26863,1,-1
6,rate_6,user_0,smallShop_28089,3,1
7,rate_7,user_0,MgisSummerStreet_142,4,1
8,rate_8,user_0,smallShop_16715,3,1
9,rate_9,user_0,smallShop_4348,3,1


In [41]:
from surprise.model_selection import KFold
from surprise import Reader, Dataset
import pandas as pd
import numpy as np

reader = Reader(rating_scale = (1, 5))

data_result = Dataset.load_from_df(data[['user_id', 'place_id', 'score']], reader)

# split data into 5 folds

data_result.split(n_folds=10)

# evaluation

from surprise import SVD, evaluate
from surprise import NMF, KNNBasic

# svd
svd = SVD()
# evaluate(svd, data_result, measures=['RMSE', 'MAE'])

# knn
knn = KNNBasic()
# evaluate(knn, data_result, measures=['RMSE', 'MAE'])

# nmf
nmf = NMF()
# evaluate(nmf, data_result, measures=['RMSE', 'MAE'])

# prediction
# user_0	smallShop_5645	2
test_user = 'user_1'
test_id = 'smallShop_7089'
real_score = 4

trainset = data_result.build_full_trainset()

# svd
svd.train(trainset)
print(svd.predict(test_user, test_id, real_score))

# knn
knn.train(trainset)
print(knn.predict(test_user, test_id, real_score))

# nmf
nmf.train(trainset)
print(nmf.predict(test_user, test_id, real_score))


# bsl_options = {
#     'method':'als',
#     'n_epochs':5,
#     'reg_u':12,
#     'reg_i':5
# }
# algo = surprise.BaselineOnly(bsl_options = bsl_options)

# np.random.seed(0)
# acc = np.zeros(3)
# cv = KFold(3)
# for i, (trainset, testset) in enumerate(cv.split(data)):
#     algo.fit(trainset)
#     predictions = algo.test(testset)
#     acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
# acc.mean()

C:\Development Tools\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


user: user_1     item: smallShop_7089 r_ui = 4.00   est = 2.96   {'was_impossible': False}
Computing the msd similarity matrix...
Done computing similarity matrix.
user: user_1     item: smallShop_7089 r_ui = 4.00   est = 3.02   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: user_1     item: smallShop_7089 r_ui = 4.00   est = 3.02   {'was_impossible': True, 'reason': 'User and item are unkown.'}


In [37]:
abc = 'abc'

In [38]:
abc.lower()

'abc'

In [12]:
import sqlite3
from random import *
from datetime import datetime
import numpy as np
# from openAPI import DBConnect

class RDBConnect():
    def drop_table(self, table):

        # user table
        self.conn.execute("""
            drop table {0}
        """.format(table))
        
    def make_user_table(self):
        
        
        ran_age_list = np.random.choice([10, 20, 30, 40, 50, 60, 70], size=1000, p=[.0262, .4832, .2612, .148, .0584, .0155, .0075])
        ran_sex_list = np.random.choice(['남성', '여성'], size=1000, p=[.62, .38])
        # print(ran_age_list)
        for i in range(1000):
            user_id = i

            ran_age = randint(ran_age_list[i], ran_age_list[i]+9)
            # 연령대는?
            #
            ran_email = "user_"+str(i)+"""@gmail.com"""
            pwd = "1234"

            ran_sex = ran_sex_list[i]
            ran_phone = "010-"+str(randint(1000, 9999))+'-'+str(randint(1000, 9999))
            query = """insert into "user" (user_id, email, password, age, sex, phone_num) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
            .format(user_id, ran_email, pwd, ran_age, ran_sex, ran_phone)
        #     print(query)
            self.cur.execute(
                query
            )

    def make_rate_table(self):

        # key list
        key_list = self.db_conn.get_item_key()
        concat_key_list = []
        for item in key_list:
            concat_key_list.append(item['cat_id'])

        rate_id = 0
        ran_score_list = np.random.choice([1, 2, 3, 4, 5], size=1000, p=[.05, .10, .7, .10, .05])
        # print(ran_score_list)
        for i in range(1000):
            ran_range = randint(1, 100)
            for j in range(ran_range):
                ran_score = ran_score_list[randint(0, 999)]
                place_id = sample(concat_key_list, 1)[0]
                if ran_score >= 3:
                    ran_review = 1
                else:
                    ran_review = -1
                query = """insert into "rate" (rate_id, user_id, place_id, date, score, review) values('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"""\
                .format("rate_"+str(rate_id), "user_"+str(i), place_id, datetime.now(), ran_score, ran_review)

                self.cur.execute(
                    query
                )
                # 평가 개수 증가
        #         db_conn.set_item_score(place_id)

                # 리뷰 점수 증가
        #         db_conn.set_item_review(place_id, ran_review)
                rate_id += 1
    def __init__(self):

        # mongoDB 연결
        self.db_conn = DBConnect()

        # sqlite 연결
        self.conn = sqlite3.connect("testDB.db")
        self.cur = self.conn.cursor()

        self.cur.execute(
        """
         create table if not exists "user" (
            "user_id" int primary key not null ,
            "email" text not null,
            "password" text not null,
            "age" int,
            "sex" text not null,
            "phone_num" text
            )
        """
        )

        self.cur.execute(
        """
         create table if not exists "rate" (
            "rate_id" int primary key not null ,
            "user_id" int references user(user_id),
            "place_id" text not null,
            "date" text not null,
            "score" int not null ,
            "review" int not null
            )
        """
        )
        self.make_user_table()
        self.make_rate_table()
        




In [13]:
test = RDBConnect()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [10]:
db_conn = DBConnect()

--INIT MONGO DB CONNECTION--
--CLIENT : MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)--
--DB : Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin')--
--ITEM : Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'admin'), 'item')--


In [27]:
client = MongoClient("127.0.0.1", 27017)
db = client.get_database("OpenAPIWithPos")
item = db['csv_small_shop']

In [28]:
result = item.find({})

In [29]:
result = list(result)

In [30]:
result = pd.DataFrame(result)

In [31]:
result.to_excel('map_db.xlsx', sheet_name='sheet1')